# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [56]:
# import libraries
import nltk
import numpy as np
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\q432892\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\q432892\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql ('SELECT * FROM DisasterResponse', engine)
X = df.message
y = df[df.columns[4:]]
category_names = y.columns
display (y.head (n=5))

,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)","('search_and_rescue',)","('security',)","('military',)","('child_alone',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [58]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [59]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001D4EB1EAE50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [61]:
y_pred = pipeline.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]), '____________________')

('related',) :
              precision    recall  f1-score   support

           0       0.75      0.28      0.41      1518
           1       0.81      0.97      0.88      4982
           2       0.47      0.15      0.23        54

    accuracy                           0.80      6554
   macro avg       0.68      0.47      0.50      6554
weighted avg       0.79      0.80      0.77      6554
 ____________________
('request',) :
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5480
           1       0.89      0.42      0.57      1074

    accuracy                           0.90      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.90      0.90      0.88      6554
 ____________________
('offer',) :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.76      0.88      0.82      3865
           1       0.78      0.60      0.68      2689

    accuracy                           0.77      6554
   macro avg       0.77      0.74      0.75      6554
weighted avg       0.77      0.77      0.76      6554
 ____________________
('medical_help',) :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6032
           1       0.65      0.05      0.10       522

    accuracy                           0.92      6554
   macro avg       0.79      0.53      0.53      6554
weighted avg       0.90      0.92      0.89      6554
 ____________________
('medical_products',) :
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6236
           1       0.76      0.05      0.09       318

    accuracy                           0.95      6554
   macro avg       0.86      0.52      0.5

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6277
           1       0.00      0.00      0.00       277

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.94      6554
 ____________________
('weather_related',) :
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      4714
           1       0.87      0.59      0.70      1840

    accuracy                           0.86      6554
   macro avg       0.87      0.78      0.81      6554
weighted avg       0.86      0.86      0.85      6554
 ____________________
('floods',) :
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6003
           1       0.94      0.34      0.50       551

    accuracy                           0.94      6554
   macro avg       0.94      0.67      0.73      

### 6. Improve your model
Use grid search to find better parameters. 

In [62]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001D4EB1EAE50>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001D4EB1EAE50>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [65]:
parameters = {
    'clf__estimator__n_estimators': [20, 50],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

In [66]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  9.8min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001D4EB1EAE50>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=4, param_grid={'clf__estimator__n_estimators': [20, 50]},
             verbose=2)

In [69]:
# Let's find the best params using our grip search
print(cv.best_params_)

{'clf__estimator__n_estimators': 50}


In [70]:
# Let's built a improved model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001D4EB1EAE50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=50)))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [71]:
y_pred = optimised_model.predict(X_test)

for i in range(30):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]), '________________________')

('related',) :
              precision    recall  f1-score   support

           0       0.74      0.29      0.42      1518
           1       0.81      0.97      0.88      4982
           2       0.48      0.20      0.29        54

    accuracy                           0.80      6554
   macro avg       0.68      0.49      0.53      6554
weighted avg       0.79      0.80      0.77      6554
 ________________________
('request',) :
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5480
           1       0.88      0.40      0.55      1074

    accuracy                           0.89      6554
   macro avg       0.89      0.70      0.75      6554
weighted avg       0.89      0.89      0.88      6554
 ________________________
('offer',) :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                       

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


________________________
('aid_related',) :
              precision    recall  f1-score   support

           0       0.76      0.88      0.82      3865
           1       0.78      0.60      0.68      2689

    accuracy                           0.77      6554
   macro avg       0.77      0.74      0.75      6554
weighted avg       0.77      0.77      0.76      6554
 ________________________
('medical_help',) :
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6032
           1       0.70      0.07      0.13       522

    accuracy                           0.92      6554
   macro avg       0.81      0.53      0.54      6554
weighted avg       0.91      0.92      0.89      6554
 ________________________
('medical_products',) :
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6236
           1       0.65      0.05      0.09       318

    accuracy                           0.95 

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6472
           1       0.00      0.00      0.00        82

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554
 ________________________
('other_infrastructure',) :
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6277
           1       0.00      0.00      0.00       277

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.94      6554
 ________________________
('weather_related',) :
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      4714
           1       0.87      0.59      0.70      1840

    accuracy                           0.86      6554
   macro avg       0.86    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
pipeline_further = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])

parameters_further = {
    'clf__estimator__learning_rate': [0.1, 0.3],
    'clf__estimator__n_estimators': [50, 100]
}

cv_further = GridSearchCV(estimator=pipeline_further, param_grid=parameters_further, cv=3, scoring='f1_weighted', verbose=3)

In [23]:
cv_further.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, score=0.5307784715093371, total= 1.6min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, score=0.5279514568442124, total= 1.6min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 


...

[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=100, score=0.5484823301715813, total= 2.9min
[CV] clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=100 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=100, score=0.5526350625381489, total= 3.0min


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 29.4min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.1, 0.3], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=3)

In [24]:
cv_further.best_params_

{'clf__estimator__learning_rate': 0.3, 'clf__estimator__n_estimators': 100}

In [36]:
optimised_model2 = cv_further.best_estimator_
print (cv_further.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...r='best'),
          learning_rate=0.3, n_estimators=100, random_state=None),
           n_jobs=1))])


In [41]:
y_pred = optimised_model2.predict(X_test)

for i in range(30):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]), '________________________')

search_and_rescue :
             precision    recall  f1-score   support

          0       0.98      0.87      0.92      6380
          1       0.09      0.46      0.15       174

avg / total       0.96      0.86      0.90      6554
 ________________________
security :
             precision    recall  f1-score   support

          0       0.99      0.79      0.88      6441
          1       0.04      0.46      0.07       113

avg / total       0.97      0.79      0.87      6554
 ________________________
military :
             precision    recall  f1-score   support

          0       0.99      0.94      0.97      6350
          1       0.30      0.75      0.42       204

avg / total       0.97      0.94      0.95      6554
 ________________________
child_alone :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554

avg / total       1.00      1.00      1.00      6554
 ________________________
water :
             precision    rec

### 9. Export your model as a pickle file

In [73]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.